In [ ]:
# Set device
import os
import torch
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim 
import torchvision.transforms as transforms
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import sklearn.metrics as metrics

In [ ]:
# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

In [ ]:
pre_trained_model = models.resnet101(pretrained=True)

for param in pre_trained_model.parameters():
    param.requires_grad = False

n_inputs = pre_trained_model.fc.in_features
classifier = nn.Sequential (nn.Linear(n_inputs, 2048),
                            nn.ReLU(),
                            nn.Dropout(p = 0.4),
                            nn.Linear(2048,2048),
                            nn.ReLU(),
                            nn.Dropout(p = 0.4),
                            nn.Linear(2048,4),
                            nn.LogSoftmax(dim = 1))

pre_trained_model.classifier = classifier
pre_trained_model.to(device)

#Visualizing the architecture of the model

#Changing the last few layers of the model to suit our data

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# optimizer + Training the Network
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(pre_trained_model.parameters(), lr=learning_rate)

for epochs in range(num_epochs):
  losses = []

  for batch_id, (data, targets) in enumerate(trainDataloader):
    data = data.to(device = device)
    targets = targets.to(device = device)

    scores = pre_trained_model(data)
    loss = criterion(scores, targets)

    losses.append(loss.item())

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()

  print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")


#Hyperparameter Tuning
#Visualizing the Training - ROC Curve, Accuracy, Sensitivity, F1 Score

In [ ]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

In [ ]:
 def check_f1_score()